In [8]:
import platform
from datetime import datetime

import psutil
import GPUtil
from tabulate import tabulate

def get_size(bytes, suffix="B"):
    """
    Scale bytes to its proper format
    e.g:
        1253656 => '1.20MB'
        1253656678 => '1.17GB'
    """
    factor = 1024
    for unit in ["", "K", "M", "G", "T", "P"]:
        if bytes < factor:
            return f"{bytes:.2f}{unit}{suffix}"
        bytes /= factor

        
print("="*40, "System Information", "="*40)
uname = platform.uname()
print(f"System: {uname.system}")
print(f"Node Name: {uname.node}")
print(f"Release: {uname.release}")
print(f"Version: {uname.version}")
print(f"Machine: {uname.machine}")
print(f"Processor: {uname.processor}")

print(uname)

======================================== System Information ========================================
System: Windows
Node Name: DESKTOP-DMA7OK6
Release: 10
Version: 10.0.19042
Machine: AMD64
Processor: Intel64 Family 6 Model 60 Stepping 3, GenuineIntel
uname_result(system='Windows', node='DESKTOP-DMA7OK6', release='10', version='10.0.19042', machine='AMD64')


In [2]:
print("="*40, "Boot Time", "="*40)
boot_time_timestamp = psutil.boot_time()
bt = datetime.fromtimestamp(boot_time_timestamp)
print(f"Boot Time: {bt.year}/{bt.month}/{bt.day} {bt.hour}:{bt.minute}:{bt.second}")

======================================== Boot Time ========================================
Boot Time: 2021/6/18 17:33:11


In [9]:
# let's print CPU information
print("="*40, "CPU Info", "="*40)
# number of cores
print("Physical cores:", psutil.cpu_count(logical=False))
print("Total cores:", psutil.cpu_count(logical=True))
# CPU frequencies
cpufreq = psutil.cpu_freq()
print(f"Max Frequency: {cpufreq.max:.2f}Mhz")
print(f"Min Frequency: {cpufreq.min:.2f}Mhz")
print(f"Current Frequency: {cpufreq.current:.2f}Mhz")
# CPU usage
print("CPU Usage Per Core:")
for i, percentage in enumerate(psutil.cpu_percent(percpu=True, interval=1)):
    print(f"Core {i}: {percentage}%")
print(f"Total CPU Usage: {psutil.cpu_percent()}%")

======================================== CPU Info ========================================
Physical cores: 4
Total cores: 8
Max Frequency: 3401.00Mhz
Min Frequency: 0.00Mhz
Current Frequency: 800.00Mhz
CPU Usage Per Core:
Core 0: 0.0%
Core 1: 0.0%
Core 2: 1.6%
Core 3: 0.0%
Core 4: 0.0%
Core 5: 3.1%
Core 6: 3.1%
Core 7: 6.3%
Total CPU Usage: 0.6%


In [11]:
# Memory Information
print("="*40, "Memory Information", "="*40)
# get the memory details
svmem = psutil.virtual_memory()
print(f"Total: {get_size(svmem.total)}")
print(f"Available: {get_size(svmem.available)}")
print(f"Used: {get_size(svmem.used)}")
print(f"Percentage: {svmem.percent}%")
print("="*20, "SWAP", "="*20)
# get the swap memory details (if exists)
swap = psutil.swap_memory()
print(f"Total: {get_size(swap.total)}")
print(f"Free: {get_size(swap.free)}")
print(f"Used: {get_size(swap.used)}")
print(f"Percentage: {swap.percent}%")

======================================== Memory Information ========================================
Total: 15.94GB
Available: 8.14GB
Used: 7.80GB
Percentage: 49.0%
==================== SWAP ====================
Total: 18.31GB
Free: 6.00GB
Used: 12.31GB
Percentage: 67.2%


In [12]:
# Disk Information
print("="*40, "Disk Information", "="*40)
print("Partitions and Usage:")
# get all disk partitions
partitions = psutil.disk_partitions()
for partition in partitions:
    print(f"=== Device: {partition.device} ===")
    print(f"  Mountpoint: {partition.mountpoint}")
    print(f"  File system type: {partition.fstype}")
    try:
        partition_usage = psutil.disk_usage(partition.mountpoint)
    except PermissionError:
        # this can be catched due to the disk that
        # isn't ready
        continue
    print(f"  Total Size: {get_size(partition_usage.total)}")
    print(f"  Used: {get_size(partition_usage.used)}")
    print(f"  Free: {get_size(partition_usage.free)}")
    print(f"  Percentage: {partition_usage.percent}%")
# get IO statistics since boot
disk_io = psutil.disk_io_counters()
print(f"Total read: {get_size(disk_io.read_bytes)}")
print(f"Total write: {get_size(disk_io.write_bytes)}")

======================================== Disk Information ========================================
Partitions and Usage:
=== Device: C:\ ===
  Mountpoint: C:\
  File system type: NTFS
  Total Size: 237.87GB
  Used: 101.71GB
  Free: 136.16GB
  Percentage: 42.8%
=== Device: D:\ ===
  Mountpoint: D:\
  File system type: NTFS
  Total Size: 596.18GB
  Used: 30.92GB
  Free: 565.25GB
  Percentage: 5.2%
Total read: 39.34GB
Total write: 23.33GB


In [13]:
# Network information
print("="*40, "Network Information", "="*40)
# get all network interfaces (virtual and physical)
if_addrs = psutil.net_if_addrs()
for interface_name, interface_addresses in if_addrs.items():
    for address in interface_addresses:
        print(f"=== Interface: {interface_name} ===")
        if str(address.family) == 'AddressFamily.AF_INET':
            print(f"  IP Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast IP: {address.broadcast}")
        elif str(address.family) == 'AddressFamily.AF_PACKET':
            print(f"  MAC Address: {address.address}")
            print(f"  Netmask: {address.netmask}")
            print(f"  Broadcast MAC: {address.broadcast}")
# get IO statistics since boot
net_io = psutil.net_io_counters()
print(f"Total Bytes Sent: {get_size(net_io.bytes_sent)}")
print(f"Total Bytes Received: {get_size(net_io.bytes_recv)}")

======================================== Network Information ========================================
=== Interface: Ethernet ===
=== Interface: Ethernet ===
  IP Address: 192.168.0.16
  Netmask: 255.255.255.0
  Broadcast IP: None
=== Interface: Ethernet ===
=== Interface: vEthernet (WSL) ===
=== Interface: vEthernet (WSL) ===
  IP Address: 172.22.32.1
  Netmask: 255.255.240.0
  Broadcast IP: None
=== Interface: vEthernet (WSL) ===
=== Interface: Loopback Pseudo-Interface 1 ===
  IP Address: 127.0.0.1
  Netmask: 255.0.0.0
  Broadcast IP: None
=== Interface: Loopback Pseudo-Interface 1 ===
Total Bytes Sent: 144.98MB
Total Bytes Received: 657.52MB


In [26]:
# GPU information
print("="*40, "GPU Details", "="*40)
gpus = GPUtil.getGPUs()
list_gpus = []
for gpu in gpus:
    # get the GPU id
    gpu_id = gpu.id
    # name of GPU
    gpu_name = gpu.name
    # get % percentage of GPU usage of that GPU
    gpu_load = f"{gpu.load*100}%"
    # get free memory in MB format
    gpu_free_memory = f"{gpu.memoryFree}MB"
    # get used memory
    gpu_used_memory = f"{gpu.memoryUsed}MB"
    # get total memory
    gpu_total_memory = f"{gpu.memoryTotal}MB"
    # get GPU temperature in Celsius
    gpu_temperature = f"{gpu.temperature} °C"
    gpu_uuid = gpu.uuid
    list_gpus.append((
        gpu_id, gpu_name, gpu_load, gpu_free_memory, gpu_used_memory,
        gpu_total_memory, gpu_temperature, gpu_uuid
    ))

print(tabulate(list_gpus, headers=("id", "name", "load", "free memory", "used memory", "total memory",
                                   "temperature", "uuid")))

======================================== GPU Details ========================================
  id  name             load    free memory    used memory    total memory    temperature    uuid
----  ---------------  ------  -------------  -------------  --------------  -------------  ----------------------------------------
   0  GeForce GTX 970  90.0%   469.0MB        3627.0MB       4096.0MB        70.0 °C        GPU-c2a46ebb-7689-71f9-0239-9c1e394d1244


try:
    process.kill()
except OSError:
    pass